In [3]:
# Importando bibliotecas
from functions import *
import pandas as pd
import locale
from pathlib import Path
from datetime import datetime
import duckdb
import gc
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import warnings
import logging
import shutil

logging.basicConfig(level=logging.WARNING, format='%(message)s')

warnings.filterwarnings("ignore")

timer = Temporizador()
timer.iniciar()

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()

pasta_input_parquet = caminho_base.parent / '01_INPUT_PIPELINE/01_BD_PARQUET'
arquivo_input_regras_negocio = caminho_base.parent / '01_INPUT_PIPELINE/02_REGRAS_NEGOCIO/KRONA_REGRAS.xlsm'
pasta_staging_parquet = caminho_base.parent / '02_STAGING_PARQUET' # Armazena arquivos parquet com tratamentos, aplicações de regras, depara, etc
pasta_input_painel = caminho_base.parent / '03_INPUT_PAINEL' # Armazena arquivos que serão consumidos no painel de S&OP para os gerentes
pasta_painel = caminho_base.parent / '05_PAINEL'

# Eliminar arquivos das pastas de 02_STAGING_PARQUET e 03_INPUT_PAINEL que serão regenerados
pastas_para_limpar = [
    pasta_staging_parquet,
    pasta_input_painel,
]

for pasta in pastas_para_limpar:
    if pasta.exists() and pasta.is_dir():
        for item in pasta.iterdir():
            if item.is_file() or item.is_symlink():
                item.unlink()
            elif item.is_dir():
                shutil.rmtree(item)

print("✅ Mapeamento de pastas concluído com sucesso!")

✅ Mapeamento de pastas concluído com sucesso!


In [27]:
# Carregar dados arquivo KRONA_REGRAS
caminho_arquivo = arquivo_input_regras_negocio

#-----------------------------------------------------------------------#
#--------------- Carregar produtos eliminar ----------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'PRODUTOS_ELIMINAR'
df_produtos_eliminar = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'COD_PROD': str})
df_produtos_eliminar['COD_PROD'] = df_produtos_eliminar['COD_PROD'].astype(str)
df_produtos_eliminar = df_produtos_eliminar.drop_duplicates(subset=['COD_PROD'])
df_produtos_eliminar = df_produtos_eliminar[df_produtos_eliminar['COD_PROD'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar Regionais Gestor ------------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'REGIONAIS_GESTOR'
df_regionais_gestor = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_regionais_gestor = df_regionais_gestor.drop_duplicates(subset=['REGIONAL', 'REGIONAL_GESTOR'])
df_regionais_gestor = df_regionais_gestor[df_regionais_gestor['REGIONAL'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carrregar Demanda Lançamento Novos Produtos ------------#
#-----------------------------------------------------------------------#
guia_excel = 'PRODUTOS_LANCAMENTOS'
df_produtos_lancamento = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
# 🚨 VALIDAR SE EXISTEM DADOS
if df_produtos_lancamento.empty:
    raise ValueError(
        "❌ ERRO: Nenhuma informação foi encontrada na aba PRODUTOS_LANCAMENTOS.\n"
        "➡️ Verifique se a planilha possui dados válidos antes de executar o pipeline."
    )
df_produtos_lancamento['JANELA LANÇAMENTO'] = df_produtos_lancamento['JANELA LANÇAMENTO'].astype(str).str.strip()
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['JANELA LANÇAMENTO'] != ''].reset_index(drop=True)
df_produtos_lancamento.rename(columns={'COD': 'COD_PROD'}, inplace=True)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['COD_PROD'].notna()].reset_index(drop=True)
df_produtos_lancamento['COD_PROD'] = df_produtos_lancamento['COD_PROD'].astype(str)

# Identifica colunas com datas válidas
col_datas = []
for col in df_produtos_lancamento.columns:
    try:
        pd.to_datetime(col, dayfirst=True, errors='raise')
        col_datas.append(col)
    except (ValueError, TypeError):
        continue

colunas_validas = ['COD_PROD'] + \
                    [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)] + \
                    col_datas
df_produtos_lancamento = df_produtos_lancamento[[col for col in colunas_validas if col in df_produtos_lancamento.columns]]

# Transforma datas em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in col_datas],
    value_vars=col_datas,
    var_name='PERIODO',
    value_name='VALOR'
)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['VALOR'].notna()].reset_index(drop=True)

# Multiplica colunas CD pelo VALOR
colunas_cd = [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)]
for col in colunas_cd:
    df_produtos_lancamento[col] = df_produtos_lancamento[col] * df_produtos_lancamento['VALOR']
df_produtos_lancamento.drop(columns=['VALOR'], inplace=True)

# Transforma colunas CD em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in colunas_cd],
    value_vars=colunas_cd,
    var_name='CD',
    value_name='QTD'
)

#-----------------------------------------------------------------------#
#---------------Carregar Regionais Construtora -------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'REGIONAIS_CONSTRUTORA'
df_regionais_construtora = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_regionais_construtora = df_regionais_construtora.drop_duplicates(subset=['REGIONAL BASE', 'REGIONAL ATUALIZADA'])

#-----------------------------------------------------------------------#
#---------------Carregar Clientes para planejamento de Demanda----------#
#-----------------------------------------------------------------------#
guia_excel = 'CLIENTES_DEMANDA'
df_clientes_plan_demanda = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'Cod_Grupo_Cliente': str})
df_clientes_plan_demanda = df_clientes_plan_demanda.drop_duplicates(subset=['Cod_Grupo_Cliente'])
df_clientes_plan_demanda = df_clientes_plan_demanda[df_clientes_plan_demanda['Cod_Grupo_Cliente'].notna()].reset_index(drop=True)

# Converter a coluna de clientes para set para acelerar o isin
lista_clientes_plan_demanda = set(df_clientes_plan_demanda['Cod_Grupo_Cliente'])

# Unir COD_PROD de df_produtos_lancamento e df_produtos_eliminar, formar uma unica lista de produtos a eliminar, e remover do df_fato_vendas_krona
# produtos_a_eliminar = pd.concat([df_produtos_eliminar[['COD_PROD']], df_produtos_lancamento[['COD_PROD']]]).drop_duplicates().reset_index(drop=True)
# FIXME: Retirei os produtos de lançamento da lista de exclusão conforme solicitação da Anna no WORD
produtos_a_eliminar = df_produtos_eliminar[['COD_PROD']].drop_duplicates().reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar DIRECIONA_CLIENTES_REGIONAL--------------------#
#-----------------------------------------------------------------------#
guia_excel = 'DIRECIONA_CLIENTES_REGIONAL'
df_direc_cli_regional = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'COD_GRUPO_CLIENTE': str, 'COD_CLIENTE': str})
df_direc_cli_regional = df_direc_cli_regional[df_direc_cli_regional['COD_CLIENTE'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar PERIODO_PREVISAO-------------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'PERIODO_PREVISAO'
df_periodo_previsao = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_periodo_previsao = df_periodo_previsao[df_periodo_previsao['PERIODO_PROJECAO'].notna()].reset_index(drop=True)
df_periodo_previsao = df_periodo_previsao.drop_duplicates(subset=['PERIODO_PROJECAO'])

print("✅ Importação e tratamento de dados do arquivo KRONA_REGRAS, concluídos com sucesso!")

✅ Importação e tratamento de dados do arquivo KRONA_REGRAS, concluídos com sucesso!


In [28]:
# Script para eliminar duplicação de Chv_Cliente no Dim_Clientes_Krona, conforme orientado por Marcos TI, criamos essa rotina para encontrar as duplicações, eliminar e gerar novo Parquet sem duplicações.

# Carregar o Parquet
df_dim_cli_krona = pd.read_parquet(pasta_input_parquet / "Dim_Clientes_Krona.parquet")

# Eliminar duplciações mantendo a primeira ocorrência
df_dim_cli_krona = df_dim_cli_krona.drop_duplicates(subset=["Chv_Cliente"], keep='first').reset_index(drop=True)

# Gerar novo Parquet sem duplicações
df_dim_cli_krona.to_parquet(pasta_input_parquet / "Dim_Clientes_Krona.parquet", index=False)

del df_dim_cli_krona
gc.collect()

0

In [ ]:
# # FIXME Cosulta para validação de dados Anna, direto do parquet Fato_Vendas_Krona.parquet, sem relacionamentos, agrupando por Cod_Produto, criando coluna mes pela Dat_Entrega_Venda no formato AAAAMM, somando Qtd_Venda

# fato_vendas = (pasta_input_parquet / "Fato_Vendas_Krona.parquet").as_posix()

# query = f"""
# WITH base AS (
#   SELECT
#     Cod_Produto,
#     Qtd_Venda,
#     CAST(Dat_Entrega_Venda AS VARCHAR) AS dt_str
#   FROM parquet_scan('{fato_vendas}')
#   WHERE TRY_CAST(NULLIF(TRIM(Cod_Bloqueio), '') AS INTEGER) IN (80,90,95,99)
#     AND Cod_Empresa IN ('01','05','08','0802','10')
# ),
# parse AS (
#   SELECT
#     Cod_Produto,
#     Qtd_Venda,
#     COALESCE(
#       TRY_STRPTIME(dt_str, '%Y-%m-%d'),
#       TRY_STRPTIME(dt_str, '%Y-%m-%d %H:%M:%S'),
#       TRY_STRPTIME(dt_str, '%d/%m/%Y'),
#       TRY_STRPTIME(dt_str, '%d/%m/%Y %H:%M:%S'),
#       TRY_STRPTIME(dt_str, '%Y%m%d')
#     )::DATE AS dt
#   FROM base
# )
# SELECT
#   Cod_Produto,
#   STRFTIME(dt, '%Y%m') AS Mes,
#   SUM(Qtd_Venda) AS Qtd_Venda_Total
# FROM parse
# WHERE dt IS NOT NULL
# GROUP BY
#   Cod_Produto,
#   Mes
# ORDER BY
#   Cod_Produto,
#   Mes
# """


# df_validacao_anna = duckdb.query(query).to_df()

# # Gerar Excel para Anna
# caminho_excel_saida = pasta_painel / f"VALIDACAO_ANNA_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
# df_validacao_anna.to_excel(caminho_excel_saida, index=False)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
# Where Des_Origem = 'Krona'
#   And Cod_Empresa IN ('01','05','08','0802','10')
#   And Dat_Emissao_Venda >= '01/01/2018';

# Na dimensão de Produto tem o seguinte filtro:
# Where Cod_Empresa IN ('01','05','08','0802','10')

# Nos casos de pedidos cancelados, consideramos apenas o que foi realmente vendido ao cliente:
# Se o pedido foi cancelado por completo, não aparece nenhuma venda.
# Se apenas uma parte foi cancelada, consideramos somente a parte que foi vendida.
# Além disso, também existem os bloqueios de pedidos, que representam a “etapa” em que o pedido se encontra. Nesses casos, é importante definir quais bloqueios devem ser considerados nessa análise.

# falei com o André aqui pelo chat, e verificou com a Aline
# a principio utiliza os códigos de bloqueio 80, 90 , 95 e 99

# No arquivo parquet, precisamos filtrar o campo Des_Origem = "Krona". O motivo é que existem pedidos faturados pelo Protheus que contam no resultado da Viqua, e este aplicativo "CML - Vendas" do Qlik Sense traz apenas os pedidos que geram resultado para a Krona

# Conslidando as informações de fontes em Parquet
empresa = 'Krona'
fact = (pasta_input_parquet / "Fato_Vendas_Krona.parquet").as_posix()
prod = (pasta_input_parquet / "Dim_Produtos_Vendas_Krona.parquet").as_posix()
cli  = (pasta_input_parquet / "Dim_Clientes_Krona.parquet").as_posix()
vend = (pasta_input_parquet / "Dim_Vendedores_Krona.parquet").as_posix()

# Eliminar produtos das listas em Excel: PRODUTOS ELIMINAR e PRODUTOS LANÇAMENTOS
duckdb.register("elim", produtos_a_eliminar[['COD_PROD']])

sql = f"""
WITH
fato AS (
  SELECT
    Cod_Produto,
    Chv_Cliente,
    Chv_Vendedor,
    DATE_TRUNC(
      'month',
      CAST(
        COALESCE(
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
        ) AS DATE
      )
    ) AS PERIODO,
    TRIM(Nom_Empresa) AS EMPRESA,
    SUM(TRY_CAST(Qtd_Venda AS DOUBLE)) AS QTD_VENDA,
    SUM(TRY_CAST(Qtd_Peso_Venda AS DOUBLE)) AS VOL_VENDA
  FROM parquet_scan('{fact}')
  WHERE UPPER(TRIM(Nom_Empresa)) LIKE '%{empresa.strip().upper()}%'
    AND UPPER(TRIM(Des_Origem))  LIKE '%{empresa.strip().upper()}%'
    AND Cod_Empresa IN ('01','05','08','0802','10')
    AND TRY_CAST(NULLIF(TRIM(Cod_Bloqueio), '') AS INTEGER) IN (80,90,95,99)
    AND TRY_CAST(Qtd_Venda AS DOUBLE) > 0
    AND Dat_Entrega_Venda IS NOT NULL
    AND TRIM(Dat_Entrega_Venda) <> ''
    AND COALESCE(
      TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
      TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
    ) >= DATE '2022-01-01'
    -- EXCLUI MÊS ATUAL
    AND DATE_TRUNC(
      'month',
      CAST(
        COALESCE(
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
        ) AS DATE
      )
    ) < DATE_TRUNC('month', CURRENT_DATE)
  GROUP BY Cod_Produto, Chv_Cliente, Chv_Vendedor, PERIODO, EMPRESA
),
prod AS (
  SELECT
    Cod_Produto,
    TRIM(Des_Produto) AS Des_Produto,
    Cod_Familia,
    TRIM(Des_Familia) AS Des_Familia,
    Cod_Linha,
    TRIM(Des_Linha) AS Des_Linha,
    TRIM(Nom_Empresa) AS EMPRESA,
    TRY_CAST(Num_Peso AS DOUBLE) AS PESO_UNIT
  FROM parquet_scan('{prod}')
  WHERE Des_Linha IS NOT NULL
    AND TRIM(Des_Linha) <> ''
    AND Cod_Empresa IN ('01','05','08','0802','10')
),
cli AS (
  SELECT
    Chv_Cliente,
    TRIM(Nom_Cliente) AS NOME_CLIENTE,
    TRIM(Nom_Empresa) AS EMPRESA,
    Chv_Vendedor_Cliente,
    TRIM(Des_Segmento) AS SEGMENTO,
    -- Corrige COD_GRUPO_CLIENTE: se vazio, usa COD_CLIENTE
    CASE
      WHEN TRIM(Cod_Grupo_Cliente) = '' OR Cod_Grupo_Cliente IS NULL
      THEN TRIM(SPLIT_PART(Chv_Cliente, '|', 2))
      ELSE TRIM(Cod_Grupo_Cliente)
    END AS COD_GRUPO_CLIENTE,
    -- Corrige DESC_GRUPO_E_CLIENTE: se vazio, usa NOME_CLIENTE
    CASE
      WHEN TRIM(Des_Grupo_e_Cliente) = '' OR Des_Grupo_e_Cliente IS NULL
      THEN TRIM(Nom_Cliente)
      ELSE TRIM(Des_Grupo_e_Cliente)
    END AS DESC_GRUPO_E_CLIENTE
  FROM parquet_scan('{cli}')
),
vend AS (
  SELECT
    Chv_Vendedor,
    TRIM(Des_Regiao) AS Des_Regiao
  FROM parquet_scan('{vend}')
),
base AS (
  SELECT f.*
  FROM fato f
  WHERE NOT EXISTS (
    SELECT 1 FROM elim e WHERE e.COD_PROD = f.Cod_Produto
  )
),
final AS (
  SELECT
    b.EMPRESA,
    TRIM(SPLIT_PART(c.Chv_Cliente, '|', 2)) AS COD_CLIENTE,
    c.NOME_CLIENTE,
    c.COD_GRUPO_CLIENTE,
    c.DESC_GRUPO_E_CLIENTE,
    c.SEGMENTO,
    b.Cod_Produto AS COD_PROD,
    p.Des_Produto AS DESC_PRODUTO,
    CAST(p.Cod_Familia AS VARCHAR) || ' - ' || p.Des_Familia AS FAMILIA,
    CAST(p.Cod_Linha   AS VARCHAR) || ' - ' || p.Des_Linha   AS LINHA,
    v1.Des_Regiao AS REGIAO_CLIENTE,
    v2.Des_Regiao AS REGIAO_MOVIMENTO,
    b.PERIODO,
    b.QTD_VENDA,
    b.VOL_VENDA,
    b.VOL_VENDA / b.QTD_VENDA AS PESO_UNIT
  FROM base b
  LEFT JOIN prod p ON b.Cod_Produto = p.Cod_Produto AND b.EMPRESA = p.EMPRESA
  LEFT JOIN cli  c ON b.Chv_Cliente = c.Chv_Cliente AND b.EMPRESA = c.EMPRESA
  LEFT JOIN vend v1 ON c.Chv_Vendedor_Cliente = v1.Chv_Vendedor
  LEFT JOIN vend v2 ON b.Chv_Vendedor         = v2.Chv_Vendedor
)
SELECT
  UPPER(EMPRESA) AS EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  SEGMENTO,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  PESO_UNIT,
  REGIAO_CLIENTE,
  REGIAO_MOVIMENTO,
  PERIODO,
  QTD_VENDA,
  VOL_VENDA
FROM final
"""
df_vendas_krona = duckdb.query(sql).to_df()

print("✅ Carregamento de dados concluído com sucesso!")

RuntimeError: Query interrupted

In [ ]:
# ============================================================
# 1. Criando coluna REGIONAL copiando a coluna REGIAO_CLIENTE 
#    no df_vendas_krona. 
#    Onde o segmento contém CONSTRUTORA ou INSTALADOR, buscar 
#    na tabela de regionais_construtora a regional atualizada.
# ============================================================

# Cria a tabela de de-para das regionais (já registrada no engine)
duckdb.register("vendas", df_vendas_krona)
duckdb.register("map_reg", df_regionais_construtora[['REGIONAL BASE','REGIONAL ATUALIZADA']])

sql = """
WITH base AS (
  SELECT
    v.*,
    -- Substitui valores vazios de REGIAO_CLIENTE por REGIAO_MOVIMENTO
    COALESCE(NULLIF(v.REGIAO_CLIENTE,''), v.REGIAO_MOVIMENTO) AS RC_FIX,
    UPPER(v.SEGMENTO) AS SEG_UP,
    UPPER(v.REGIAO_CLIENTE) AS RC,
    UPPER(v.REGIAO_MOVIMENTO) AS RM
  FROM vendas v
),

ajuste AS (
  SELECT
    b.*,
    CASE
      -- 1) Se SEGMENTO contém CONSTRUTORA ou INSTALADOR => usa de-para
      WHEN b.SEG_UP LIKE '%CONSTRUTORA%' OR b.SEG_UP LIKE '%INSTALADOR%'
        THEN COALESCE(m."REGIONAL ATUALIZADA", b.RC_FIX)
      -- ============================================================
      -- 2. Converter TELEVENDAS - Regras para definir REGIONAL:
      --    REGIONAL = CONSTRUTORA => REGIONAL_CONSTRUTORA
      --    REGIAO_CLIENTE = TELEVENDAS e REGIAO_MOVIMENTO = TELEVENDAS => TELEVENDAS
      --    REGIAO_CLIENTE != TELEVENDAS e REGIAO_MOVIMENTO = TELEVENDAS => TELEVENDAS
      --    REGIAO_CLIENTE = TELEVENDAS e REGIAO_MOVIMENTO != TELEVENDAS => REGIAO_MOVIMENTO
      --    Caso contrário => REGIAO_CLIENTE
      -- ============================================================
      WHEN b.RC='TELEVENDAS' AND b.RM='TELEVENDAS' THEN 'TELEVENDAS'
      WHEN b.RC<>'TELEVENDAS' AND b.RM='TELEVENDAS' THEN 'TELEVENDAS'
      WHEN b.RC='TELEVENDAS' AND b.RM<>'TELEVENDAS' THEN b.RM
      ELSE b.RC_FIX
    END AS REGIONAL
  FROM base b
  LEFT JOIN map_reg m
    ON m."REGIONAL BASE" = b.REGIAO_CLIENTE
)

-- ============================================================
-- Resultado final consolidado
-- ============================================================
SELECT
  EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  REGIONAL,
  PERIODO,
  SUM(QTD_VENDA) AS QTD_VENDA,
  SUM(VOL_VENDA) AS VOL_VENDA
FROM ajuste
WHERE REGIONAL IS NOT NULL AND REGIONAL <> ''
GROUP BY
  EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  REGIONAL,
  PERIODO
"""

# Executa no DuckDB
df_vendas_krona = duckdb.query(sql).to_df()

# Inserir REGIONAL_GESTOR no df_vendas_krona
df_vendas_krona = pd.merge(
    df_vendas_krona,
    df_regionais_gestor,
    left_on='REGIONAL',
    right_on='REGIONAL',
    how='left'
)

colunas_ordenadas = [
    "EMPRESA",
    "COD_CLIENTE",
    "NOME_CLIENTE",
    "COD_GRUPO_CLIENTE",
    "DESC_GRUPO_E_CLIENTE",
    "COD_PROD",
    "DESC_PRODUTO",
    "FAMILIA",
    "LINHA",
    "REGIONAL",
    "REGIONAL_GESTOR",
    "PERIODO",
    "QTD_VENDA",
    "VOL_VENDA"
]

df_vendas_krona = df_vendas_krona[colunas_ordenadas]

# Salvar df_vendas_krona em Parquet para salvar as alterações, filtros e regras aplicadas no histórico, otimizando memória e garantindo rastreabilidade
df_vendas_krona.to_parquet(pasta_staging_parquet / "df_vendas_krona.parquet", index=False)

print("✅ Organização de Regionais e Inserção de Regional Gestor concluídos com sucesso!")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Organização de Regionais e Inserção de Regional Gestor concluídos com sucesso!


In [ ]:
# 🦆 Exportação de Dados Vendas para Planejamento Colaborativo
# 🎯 Objetivo: Exportar CSV para o Plano Colaborativo
df_vendas_krona['NIVEL_PLAN_DEMANDA'] = np.where(
    df_vendas_krona['COD_GRUPO_CLIENTE'].isin(lista_clientes_plan_demanda),
    'CLIENTE',
    'PRODUTO'
)

# Separa os DataFrames
df_hist_vend_PRODUTO = df_vendas_krona[df_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'PRODUTO']
df_hist_vend_CLIENTE = df_vendas_krona[df_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'CLIENTE']

# Eliminar coluna NIVEL_PLAN_DEMANDA
df_hist_vend_PRODUTO = df_hist_vend_PRODUTO.drop(columns=['NIVEL_PLAN_DEMANDA'])
df_hist_vend_CLIENTE = df_hist_vend_CLIENTE.drop(columns=['NIVEL_PLAN_DEMANDA'])

# Agrupar df_hist_vend_PRODUTO por REGIONAL_GESTOR, FAMILIA, PERIODO, VOL_VENDA
df_hist_vend_PRODUTO = df_hist_vend_PRODUTO.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

# Salva como CSV
df_hist_vend_PRODUTO.to_csv(
    pasta_input_painel / 'HIST_VENDA_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

# Agrupar df_hist_vend_CLIENTE por COD_GRUPO_CLIENTE, DESC_GRUPO_E_CLIENTE, REGIONAL_GESTOR, FAMILIA, PERIODO, VOL_VENDA
df_hist_vend_CLIENTE = df_hist_vend_CLIENTE.groupby(
    ["COD_GRUPO_CLIENTE","DESC_GRUPO_E_CLIENTE", "REGIONAL_GESTOR", 'REGIONAL', "FAMILIA", "PERIODO"],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

# Salva como CSV
df_hist_vend_CLIENTE.to_csv(
    pasta_input_painel / 'HIST_VENDA_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

In [ ]:
# Gerar os arquivos com média de vendas para Planejamento Colaborativo Agregado
# Encontrar o primeiro dia do mês atual

colunas_agregadas = ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA']

hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)

# Calcular o primeiro dia do mês de 6 meses atrás (excluindo mês atual)
primeiro_dia_6_meses_atras = (primeiro_dia_mes_atual - pd.DateOffset(months=6)).to_pydatetime()

# Filtrar apenas os últimos 6 meses (excluindo mês atual)
mask = (df_hist_vend_PRODUTO['PERIODO'] >= primeiro_dia_6_meses_atras) & (df_hist_vend_PRODUTO['PERIODO'] < primeiro_dia_mes_atual)
df_hist_vend_PRODUTO_ultimos_6_meses = df_hist_vend_PRODUTO.loc[mask].copy()

# Ordenar por data crescente
df_hist_vend_PRODUTO_ultimos_6_meses = df_hist_vend_PRODUTO_ultimos_6_meses.sort_values('PERIODO').reset_index(drop=True)

# DataFrame dos 3 meses mais recentes (últimos 3 meses do intervalo filtrado)
df_3_meses_mais_recentes = df_hist_vend_PRODUTO_ultimos_6_meses.copy()

# Identificar as 3 datas mais recentes (sem duplicar por linha)
meses_recentes = sorted(df_3_meses_mais_recentes['PERIODO'].unique())[-3:]

# Filtrar todas as linhas que pertencem a esses 3 meses
df_3_meses_mais_recentes = df_3_meses_mais_recentes[df_3_meses_mais_recentes['PERIODO'].isin(meses_recentes)].copy()

# Agrupa pelas colunas desejadas e calcula a média das colunas numéricas
df_3_meses_mais_recentes_media = df_3_meses_mais_recentes.groupby(colunas_agregadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 3 MESES' na coluna
df_3_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 3 MESES'

# Agrupamento fazendo média dos 6 meses
df_6_meses_mais_recentes_media = df_hist_vend_PRODUTO_ultimos_6_meses.copy()
df_6_meses_mais_recentes_media = df_6_meses_mais_recentes_media.groupby(colunas_agregadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 6 MESES' na coluna
df_6_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 6 MESES'

# Concatenar os DataFrames
df_media_vendas_PRODUTO = pd.concat([df_3_meses_mais_recentes_media, df_6_meses_mais_recentes_media], ignore_index=True)

# Pivotar a coluna MEDIA
df_media_vendas_PRODUTO = df_media_vendas_PRODUTO.pivot_table(
    index=colunas_agregadas,
    columns='MEDIA',
    values='VOL_VENDA',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Gerar o arquivo CSV
df_media_vendas_PRODUTO.to_csv(
    pasta_input_painel / 'MEDIA_VENDA_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

In [ ]:
# Gerar os arquivos com média de vendas para Planejamento Colaborativo por Cliente
# Encontrar o primeiro dia do mês atual
colunas_agrupadas = ['COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE', 'REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA']

hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)

# Calcular o primeiro dia do mês de 6 meses atrás (excluindo mês atual)
primeiro_dia_6_meses_atras = (primeiro_dia_mes_atual - pd.DateOffset(months=6)).to_pydatetime()

# Filtrar apenas os últimos 6 meses (excluindo mês atual)
mask = (df_hist_vend_CLIENTE['PERIODO'] >= primeiro_dia_6_meses_atras) & (df_hist_vend_CLIENTE['PERIODO'] < primeiro_dia_mes_atual)
df_hist_vend_CLIENTE_ultimos_6_meses = df_hist_vend_CLIENTE.loc[mask].copy()

# Ordenar por data crescente
df_hist_vend_CLIENTE_ultimos_6_meses = df_hist_vend_CLIENTE_ultimos_6_meses.sort_values('PERIODO').reset_index(drop=True)

# DataFrame dos 3 meses mais recentes (últimos 3 meses do intervalo filtrado)
df_3_meses_mais_recentes = df_hist_vend_CLIENTE_ultimos_6_meses.copy()

# Identificar as 3 datas mais recentes (sem duplicar por linha)
meses_recentes = sorted(df_3_meses_mais_recentes['PERIODO'].unique())[-3:]

# Filtrar todas as linhas que pertencem a esses 3 meses
df_3_meses_mais_recentes = df_3_meses_mais_recentes[df_3_meses_mais_recentes['PERIODO'].isin(meses_recentes)].copy()

# Agrupa pelas colunas desejadas e calcula a média das colunas numéricas
df_3_meses_mais_recentes_media = df_3_meses_mais_recentes.groupby(colunas_agrupadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 3 MESES' na coluna
df_3_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 3 MESES'

# Agrupamento fazendo média dos 6 meses
df_6_meses_mais_recentes_media = df_hist_vend_CLIENTE_ultimos_6_meses.copy()
df_6_meses_mais_recentes_media = df_6_meses_mais_recentes_media.groupby(colunas_agrupadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 6 MESES' na coluna
df_6_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 6 MESES'

# Concatenar os DataFrames
df_media_vendas_PRODUTO = pd.concat([df_3_meses_mais_recentes_media, df_6_meses_mais_recentes_media], ignore_index=True)

# Pivotar a coluna MEDIA
df_media_vendas_PRODUTO = df_media_vendas_PRODUTO.pivot_table(
    index=colunas_agrupadas,
    columns='MEDIA',
    values='VOL_VENDA',
    aggfunc='sum',
    fill_value=0
).reset_index()


# Gerar o arquivo CSV
df_media_vendas_PRODUTO.to_csv(
    pasta_input_painel / 'MEDIA_VENDA_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

# FIXME
del df_hist_vend_PRODUTO, df_hist_vend_CLIENTE, df_vendas_krona, produtos_a_eliminar
gc.collect()

print("✅ Bases de Vendas para Planejamento Colaborativo geradas com sucesso!")

✅ Bases de Vendas para Planejamento Colaborativo geradas com sucesso!


In [ ]:
"""
🧩 Histórico dos Modelos Testados no Projeto
--------------------------------------------

1. Média Simples / Média 12M
   Descrição: cálculo da média das vendas dos últimos 12 meses.
   Objetivo: criar um ponto de partida rápido para testar estabilidade.
   Vantagem: extremamente leve e previsível.
   Limitação: ignora tendências (crescimento ou queda) e não reage a sazonalidades.

2. Média Móvel Ponderada
   Descrição: média dos últimos 12 meses com pesos maiores para os meses mais recentes.
   Objetivo: suavizar o histórico sem perder sensibilidade à tendência recente.
   Vantagem: melhora ligeiramente a resposta a movimentos recentes.
   Limitação: ainda não reconhece padrões anuais completos de sazonalidade.

3. Sazonalidade Percentual Histórica
   Descrição: calculava a participação média de cada mês no total anual.
   Objetivo: reproduzir o comportamento sazonal real da empresa.
   Vantagem: respeita picos e vales mensais do último ano completo.
   Limitação: dependente da qualidade do último ano — não projeta volume total, apenas distribui.

4. LightGBM
   Descrição: modelo de machine learning (boosting de árvores) aplicado sobre variáveis sazonais (seno/cosseno dos meses).
   Objetivo: prever volumes mensais aprendendo padrões não lineares.
   Vantagem: aprendizado rápido e robusto em bases amplas.
   Limitação: exige ajuste fino e mais dados; em séries curtas, tende a superajustar.

5. Regressão Linear (nível anual)
   Descrição: ajusta uma reta sobre as vendas anuais (y = a·x + b).
   Objetivo: capturar tendências de crescimento ou queda sustentadas.
   Vantagem: intuitivo e fácil de justificar visualmente.
   Limitação: não lida bem com oscilações bruscas ou séries curtas.

6. Holt-Winters Aditivo
   Descrição: modelo clássico de séries temporais com nível, tendência e sazonalidade (additive trend + seasonal).
   Objetivo: gerar previsões suaves mantendo padrão anual.
   Vantagem: reconhecido e equilibrado entre suavidade e tendência.
   Limitação: pesado em grandes volumes e instável em séries curtas.

7. Ensemble Estatístico (fase intermediária)
   Descrição: combinação ponderada de modelos simples (Regressão + Média + Suavização).
   Objetivo: estabilizar volumes sem perder aparência estatística.
   Vantagem: resultados consistentes e realistas.
   Limitação: apresentava sempre o mesmo nome, sem variação por empresa.
      
"""

"""
📊 Modelos de Previsão Aplicados (Versão Final)

Este script aplica cinco modelos distintos de previsão de séries temporais, compara o desempenho com base no RMSE
(Root Mean Squared Error) e seleciona automaticamente o mais assertivo para gerar a projeção final.

Modelos utilizados:

1. Regressão Linear
   - Captura: tendências lineares de longo prazo.
   - Uso: quando o histórico mostra crescimento ou queda estável.
   - Força: fácil de justificar e visualizar; ideal para projeções simples e diretas.

2. Holt-Winters (Suavização Exponencial com sazonalidade multiplicativa)
   - Captura: padrões sazonais e tendência, com maior peso para os dados mais recentes.
   - Uso: quando há sazonalidade clara e variações proporcionais ao volume.
   - Força: modelo clássico, confiável e com excelente desempenho em séries temporais mensais.

3. ARIMA (AutoRegressive Integrated Moving Average)
   - Captura: dependência temporal e ruído estatístico, sem necessidade de sazonalidade explícita.
   - Uso: quando há padrão autoregressivo e estabilidade sem sazonalidade forte.
   - Força: modelo estatístico robusto, ideal para séries estacionárias ou suavizadas.

4. Random Forest Regressor com variáveis sazonais
   - Captura: relações não lineares e interações entre tempo, mês e ano.
   - Uso: quando há sazonalidade, mas o padrão não é linear nem estável.
   - Força: flexível, adaptável e resistente a ruídos; ótimo para séries com comportamento irregular.

5. Prophet (Facebook) com sazonalidade anual
   - Captura: tendência, sazonalidade e feriados (se configurado).
   - Uso: quando há sazonalidade anual bem definida e histórico suficiente.
   - Força: fácil de ajustar, escalável e excelente para previsões com múltiplos componentes.

O modelo com menor RMSE nos últimos 12 meses é selecionado automaticamente para gerar a previsão final,
que é incorporada à coluna 'VOL_VENDA_REAL' junto ao histórico, com marcação do modelo escolhido.

"""

"\n📊 Modelos de Previsão Aplicados (Versão Final)\n\nEste script aplica cinco modelos distintos de previsão de séries temporais, compara o desempenho com base no RMSE\n(Root Mean Squared Error) e seleciona automaticamente o mais assertivo para gerar a projeção final.\n\nModelos utilizados:\n\n1. Regressão Linear\n   - Captura: tendências lineares de longo prazo.\n   - Uso: quando o histórico mostra crescimento ou queda estável.\n   - Força: fácil de justificar e visualizar; ideal para projeções simples e diretas.\n\n2. Holt-Winters (Suavização Exponencial com sazonalidade multiplicativa)\n   - Captura: padrões sazonais e tendência, com maior peso para os dados mais recentes.\n   - Uso: quando há sazonalidade clara e variações proporcionais ao volume.\n   - Força: modelo clássico, confiável e com excelente desempenho em séries temporais mensais.\n\n3. ARIMA (AutoRegressive Integrated Moving Average)\n   - Captura: dependência temporal e ruído estatístico, sem necessidade de sazonalidade

In [ ]:
parar_execucao()

In [ ]:
# PREVISAO ESTATISTICA — 5 MODELOS + MELHOR POR COD_PROD (SEM TRATAMENTO EXTRA)
print("🔄 Iniciando processo de previsão estatística...")

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA


# ============================================================
# 0) CARREGAR df_vendas_krona DO PARQUET (para limpar memória)
# ============================================================

df_vendas_krona = pd.read_parquet(pasta_staging_parquet / "df_vendas_krona.parquet")


# ============================================================
# 1) AGRUPAMENTO PADRÃO (SEU)
# ============================================================

df_group = (
    df_vendas_krona
    .groupby(["COD_PROD", "PERIODO"], as_index=False)
    .agg(VOL_VENDA=("VOL_VENDA", "sum"))
    .sort_values(["COD_PROD", "PERIODO"])
)


# ============================================================
# 2) CALENDÁRIO FUTURO (SEU) — SEM NORMALIZAR
# ============================================================

future_dates = pd.DatetimeIndex(
    df_periodo_previsao["PERIODO_PROJECAO"].drop_duplicates().sort_values()
)

if len(future_dates) == 0:
    raise ValueError("df_periodo_previsao['PERIODO_PROJECAO'] está vazio.")

primeiro_mes_previsao = future_dates.min()
ultimo_mes_hist = primeiro_mes_previsao - pd.offsets.MonthBegin(1)

df_hist_base = df_group[df_group["PERIODO"] <= ultimo_mes_hist].copy()
if df_hist_base.empty:
    raise ValueError("Histórico vazio após corte pelo calendário futuro.")

horizon = len(future_dates)


# ============================================================
# 3) MÉTRICA (WAPE ou MAPE)
# ============================================================

METRICA_USADA = "WAPE"  # "WAPE" recomendado; se quiser "MAPE", troque aqui.

def wape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return float(np.mean(np.abs(y_true - y_pred)))
    return float(np.sum(np.abs(y_true - y_pred)) / denom)

def safe_mape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = y_true != 0
    if mask.sum() == 0:
        return float(np.mean(np.abs(y_true - y_pred)))
    return float(mean_absolute_percentage_error(y_true[mask], y_pred[mask]))

def metric(y_true, y_pred) -> float:
    return wape(y_true, y_pred) if METRICA_USADA == "WAPE" else safe_mape(y_true, y_pred)


# ============================================================
# 4) MODELOS (funções simples)
# ============================================================

def pred_hw(y_train, steps):
    # tenta sazonal; se falhar, cai pra Holt trend
    try:
        m = ExponentialSmoothing(y_train, trend="add", seasonal="multiplicative", seasonal_periods=12).fit()
        return np.maximum(m.forecast(steps), 0)
    except Exception:
        try:
            m = ExponentialSmoothing(y_train, trend="add", seasonal="additive", seasonal_periods=12).fit()
            return np.maximum(m.forecast(steps), 0)
        except Exception:
            m = ExponentialSmoothing(y_train, trend="add", seasonal=None).fit()
            return np.maximum(m.forecast(steps), 0)

def pred_arima(y_train, steps):
    m = ARIMA(y_train, order=(1,1,1)).fit()
    return np.maximum(m.forecast(steps), 0)

def pred_lr(y_train, steps):
    y_train = np.asarray(y_train, dtype=float)
    t = np.arange(len(y_train)).reshape(-1, 1)
    lr = LinearRegression().fit(t, y_train)
    t_future = np.arange(len(y_train), len(y_train) + steps).reshape(-1, 1)
    return np.maximum(lr.predict(t_future), 0)

def make_features_simple(y):
    # features MINIMAS pra ML sem depender de painel:
    # usa índice de tempo + mês (do próprio período)
    return y

def pred_rf(period_index_train, y_train, period_index_pred):
    # features: time + mês + ano
    X_train = pd.DataFrame({
        "time": np.arange(len(period_index_train)),
        "mes": period_index_train.month,
        "ano": period_index_train.year
    })
    rf = RandomForestRegressor(n_estimators=400, random_state=42)
    rf.fit(X_train, y_train)

    X_pred = pd.DataFrame({
        "time": np.arange(len(period_index_train), len(period_index_train) + len(period_index_pred)),
        "mes": period_index_pred.month,
        "ano": period_index_pred.year
    })
    return np.maximum(rf.predict(X_pred), 0)

def pred_gb(period_index_train, y_train, period_index_pred):
    X_train = pd.DataFrame({
        "time": np.arange(len(period_index_train)),
        "mes": period_index_train.month,
        "ano": period_index_train.year
    })
    gb = GradientBoostingRegressor(random_state=42)
    gb.fit(X_train, y_train)

    X_pred = pd.DataFrame({
        "time": np.arange(len(period_index_train), len(period_index_train) + len(period_index_pred)),
        "mes": period_index_pred.month,
        "ano": period_index_pred.year
    })
    return np.maximum(gb.predict(X_pred), 0)


# ============================================================
# 5) ESCOLHER MELHOR MODELO POR COD_PROD (backtest) + prever futuro
# ============================================================

JANELA_VALIDACAO = 12

registros = []

for cod_prod, df_sku in df_hist_base.groupby("COD_PROD"):
    df_sku = df_sku.sort_values("PERIODO")
    y = df_sku["VOL_VENDA"].values.astype(float)
    idx = pd.DatetimeIndex(df_sku["PERIODO"])

    # histórico muito curto -> LR
    if len(y) < 6:
        fc = pred_lr(y, horizon)
        best = "LinearRegression_Fallback"
        for per, val in zip(future_dates, fc):
            registros.append([cod_prod, per, float(val), best])
        continue

    J = min(JANELA_VALIDACAO, max(3, len(y)//3))
    y_train, y_val = y[:-J], y[-J:]
    idx_train, idx_val = idx[:-J], idx[-J:]

    scores = {}

    # 1) HW
    try:
        pred_val = pred_hw(y_train, J)
        scores["HoltWinters"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 2) ARIMA
    try:
        pred_val = pred_arima(y_train, J)
        scores["ARIMA"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 3) LR
    try:
        pred_val = pred_lr(y_train, J)
        scores["LinearRegression"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 4) RF (ML)
    try:
        pred_val = pred_rf(idx_train, y_train, idx_val)
        scores["RandomForest"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 5) GB (ML)
    try:
        pred_val = pred_gb(idx_train, y_train, idx_val)
        scores["GradientBoosting"] = metric(y_val, pred_val)
    except Exception:
        pass

    # escolher melhor e treinar no histórico completo
    if not scores:
        best = "LinearRegression_Fallback"
        fc = pred_lr(y, horizon)
    else:
        best = min(scores.items(), key=lambda x: x[1])[0]

        if best == "HoltWinters":
            fc = pred_hw(y, horizon)
        elif best == "ARIMA":
            fc = pred_arima(y, horizon)
        elif best == "LinearRegression":
            fc = pred_lr(y, horizon)
        elif best == "RandomForest":
            fc = pred_rf(idx, y, future_dates)
        else:  # GradientBoosting
            fc = pred_gb(idx, y, future_dates)

    for per, val in zip(future_dates, fc):
        registros.append([cod_prod, per, float(val), best])

df_forecast = pd.DataFrame(registros, columns=["COD_PROD", "PERIODO", "VOL_VENDA_REAL", "MODELO_ESCOLHIDO"])


# ============================================================
# 6) SAÍDA FINAL (histórico + projeção)
# ============================================================

df_final_hist = df_hist_base.rename(columns={"VOL_VENDA": "VOL_VENDA_REAL"}).copy()
df_final_hist["MODELO_ESCOLHIDO"] = np.nan

df_forecast_estatistico_krona = pd.concat([df_final_hist, df_forecast], ignore_index=True)


# ============================================================
# 7) SALVAR CSV (EXATO COMO VOCÊ PEDIU)
# ============================================================

df_forecast_estatistico_krona.to_csv(
    pasta_staging_parquet / "df_forecast_estatistico_krona.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
    decimal=",",
    float_format="%.2f"
)

print("✅ Finalizado e salvo: df_forecast_estatistico_krona.csv")

🔄 Iniciando processo de previsão estatística...


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# ============================================================
# DESAGREGAÇÃO INTELIGENTE BASEADA NO MIX DOS ÚLTIMOS 6 MESES
# ============================================================

# 1) Definir data limite (6 meses atrás do último mês completo)
data_limite = ultimo_mes_completo - pd.DateOffset(months=6)

# 2) Filtrar histórico dos últimos 6 meses
df_6m = df_vendas_krona[df_vendas_krona["PERIODO"] > data_limite].copy()

# 3) Somar volume por linha completa (todas as suas chaves originais)
chaves = [
    "EMPRESA",
    "COD_CLIENTE",
    "NOME_CLIENTE",
    "COD_GRUPO_CLIENTE",
    "DESC_GRUPO_E_CLIENTE",
    "COD_PROD",
    "DESC_PRODUTO",
    "FAMILIA",
    "LINHA",
    "REGIONAL",
    "REGIONAL_GESTOR"
]

df_mix = (
    df_6m.groupby(chaves)["VOL_VENDA"]
    .sum()
    .reset_index()
)

# 4) Total geral dos últimos 6 meses (somatório de tudo)
total_6m = df_mix["VOL_VENDA"].sum()

# 5) Criar PESO_MIX com base nos últimos 6 meses
df_mix["PESO_MIX"] = df_mix["VOL_VENDA"] / total_6m

# 6) Filtrar somente forecast futuro
df_forecast_fut = df_forecast_estatistico_krona[
    df_forecast_estatistico_krona["PERIODO"] > ultimo_mes_completo
].copy()

# 7) Desagregar: multiplicar forecast mensal total pelo mix real
df_forecast_desagregado = (
    df_forecast_fut
    .assign(key=1)
    .merge(df_mix.assign(key=1), on="key")
    .drop(columns="key")
)

df_forecast_desagregado["VOL_VENDA"] = (
    df_forecast_desagregado["VOL_VENDA_REAL"] *
    df_forecast_desagregado["PESO_MIX"]
)

# 8) Selecionar colunas finais no mesmo layout do seu pipeline
df_forecast_desagregado = df_forecast_desagregado[[
    "EMPRESA",
    "COD_CLIENTE",
    "NOME_CLIENTE",
    "COD_GRUPO_CLIENTE",
    "DESC_GRUPO_E_CLIENTE",
    "COD_PROD",
    "DESC_PRODUTO",
    "FAMILIA",
    "LINHA",
    "REGIONAL",
    "REGIONAL_GESTOR",
    "PERIODO",
    "VOL_VENDA"
]]

# 9) Salvar
df_forecast_desagregado.to_parquet(
    pasta_staging_parquet / "df_forecast_vendas_krona.parquet",
    index=False,
    compression='snappy'
)

# FIXME
del df_vendas_krona_hist_mes, df_final, df_forecast_final, df_lr, df_prophet, df_vendas_krona, df
gc.collect()

0

In [ ]:
# Separar a df_forecast_vendas_krona em dois dataframes:
# df_forecast_vendas_krona_CLIENTE: clientes que terão planejamento de demanda
# df_forecast_vendas_krona_PRODUTO: produtos que terão planejamento de demanda

df_forecast_vendas_krona = df_forecast_desagregado.copy()

# Se lista_clientes_plan_demanda estiver vazio → todos são PRODUTO
if lista_clientes_plan_demanda and len(lista_clientes_plan_demanda) > 0:
    df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] = np.where(
        df_forecast_vendas_krona['COD_GRUPO_CLIENTE'].isin(lista_clientes_plan_demanda),
        'CLIENTE',
        'PRODUTO'
    )
else:
    # Se não existe cliente para plan. demanda → tudo produto
    df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] = 'PRODUTO'


# Separar os dataframes com cópia explícita
df_forecast_vendas_krona_CLIENTE = df_forecast_vendas_krona[df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'CLIENTE'].copy()
df_forecast_vendas_krona_PRODUTO = df_forecast_vendas_krona[df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'PRODUTO'].copy()

# Eliminar coluna NIVEL_PLAN_DEMANDA
df_forecast_vendas_krona_CLIENTE.drop(columns=['NIVEL_PLAN_DEMANDA'], inplace=True)
df_forecast_vendas_krona_CLIENTE.reset_index(drop=True, inplace=True)
df_forecast_vendas_krona_PRODUTO.drop(columns=['NIVEL_PLAN_DEMANDA'], inplace=True)
df_forecast_vendas_krona_PRODUTO.reset_index(drop=True, inplace=True)

# Eliminar colunas NOME_CLIENTE E DESC_GRUPO_E_CLIENTE
df_forecast_vendas_krona_PRODUTO.drop(columns=['COD_CLIENTE', 'NOME_CLIENTE', 'COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE'], inplace=True)

# Sumarizar df_forecast_vendas_krona_PRODUTO por EMPRESA, COD_PROD, DESC_PRODUTO, FAMILIA, LINHA, REGIONAL, PERIODO
df_forecast_vendas_krona_PRODUTO = df_forecast_vendas_krona_PRODUTO.groupby(
    ['EMPRESA', 'COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 'REGIONAL', 'REGIONAL_GESTOR', 'PERIODO'],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

# Trazer PESO_UNIT do parquet Dim_Produtos_Vendas_krona
# Carregar parquet Dim_Produtos_Vendas_krona
dim_produtos_vendas_krona = pd.read_parquet(
    pasta_input_parquet / "Dim_Produtos_Vendas_krona.parquet"
)

# Normalizar maiuscula coluna Nom_Empresa
def normalizar(texto):
    if pd.isna(texto):
        return ''
    return ''.join(e for e in texto.upper() if e.isalnum())

dim_produtos_vendas_krona["Nom_Empresa"] = dim_produtos_vendas_krona["Nom_Empresa"].str.upper()

# Selecionar colunas necessárias e remover duplicatas
dim_produtos_vendas_krona = dim_produtos_vendas_krona[['Cod_Produto', 'Nom_Empresa', 'Num_Peso']].drop_duplicates(subset=['Cod_Produto', 'Nom_Empresa'])

# Merge para trazer PESO_UNIT
df_forecast_vendas_krona_PRODUTO = pd.merge(
    df_forecast_vendas_krona_PRODUTO,
    dim_produtos_vendas_krona,
    how='left',
    left_on=['COD_PROD', 'EMPRESA'],
    right_on=['Cod_Produto', 'Nom_Empresa']
)

# Renomear coluna Num_Peso para PESO_UNIT
df_forecast_vendas_krona_PRODUTO.rename(columns={'Num_Peso': 'PESO_UNIT'}, inplace=True)

# Eliminar colunas desnecessárias
df_forecast_vendas_krona_PRODUTO.drop(columns=['Cod_Produto', 'Nom_Empresa'], inplace=True)

# Gerar arquivos em PARQUET
df_forecast_vendas_krona_PRODUTO.to_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_PRODUTO.parquet', index=False)
df_forecast_vendas_krona_CLIENTE.to_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_CLIENTE.parquet', index=False)

# 📤 Exportação de Dados Forecast para Planejamento Colaborativo
# 📊 Nível de agregação: REGIONAL_GESTOR, FAMILIA e PERIODO

df_Forecast_PRODUTO = df_forecast_vendas_krona_PRODUTO.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)
df_Forecast_PRODUTO.to_csv(
    pasta_staging_parquet / 'FORECAST_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

# 📤 Exportação de Dados Forecast para Planejamento Colaborativo
# 📊 Nível de agregação: REGIONAL_GESTOR, COD_GRUPO_CLIENTE, DESC_GRUPO_E_CLIENTE, FAMILIA e PERIODO

df_Forecast_CLIENTE = df_forecast_vendas_krona_CLIENTE.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)
df_Forecast_CLIENTE.to_csv(
    pasta_staging_parquet / 'FORECAST_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

# FIXME
del df_forecast_desagregado, dim_produtos_vendas_krona, df_forecast_vendas_krona_PRODUTO, df_forecast_vendas_krona_CLIENTE, lista_clientes_plan_demanda
gc.collect()

print("✅ Processamento Estatístico e Dados Forecast gerados com sucesso!")

✅ Processamento Estatístico e Dados Forecast gerados com sucesso!


In [ ]:
timer.finalizar()
print("🎯 Processo concluído com sucesso!")


⏱️ Tempo total de processamento: 2 min 51.6 s
🎯 Processo concluído com sucesso!


In [ ]:
# Pendencias
# 1. Ao final do projeto, apagar os FIXME de validação de dados que estão comentados no meio do programa
# 2. Pendencia: Retirei dados de produtos lançamentos da lista de exclusão, pois preciso fazer previsão estatística e considerar essa condição conforme solicitações da Anna no WORD
# 3. Ver modelo de Desagregação conforme implementao na VIQUA
# 4. Ver com GPT sobre dimensionamento de VM para o projeto
# 6. Verificar se todos as tabelas carregadas do arquivo KRONA_REGRAS estão sendo utilizadas no código final
# 7. No código, ao importar os dados de vendas, se o ultimo mes for o mesmo mes de hoje, retirar esse mês da base de vendas para evitar vazamento de dados
# 8. Fazer um filtro por Cliente e Regional, e verificar porque existe código de cliente com duas regionais
# 9. Corrigir endereços de pastas que foram alteradas no projeto
# 10. Como resolver a duplicação de clientes ou regionais, a princípio a duplicação de chaves foi resolvida
# 11. Criar visão resumo em KG e R$ conforme enviado modelo Anna
# 12. Criar visão resumo, após processar dados dos gerentes no pipeline, conforme modelo enviado por Anna. Esses quadros são apresentados à diretoria
# 13. Criar formato para revisão do plano de demanda
# 14. Criar um formato da exportação de dados (para conferência conforme Anna enviará o formato do arquivo)

# Ferramenta Excel
# 1. Direcionar consumo das bases considerando a nova pasta BD_PLAN_COLAB_FERR_EXCEL dentro da pasta_staging_parquet

# Pendencias resolvidas e aplicaveis as projetos VIQUA e LINEAR
# 1. No código, ao importar os dados de vendas, se o ultimo mes for o mesmo mes de hoje, retirar esse mês da base de vendas para evitar vazamento de dados

# Comando criar Requirements.txt
# pip freeze > ".\00_SCRIPTS\requirements.txt"